In [1]:
## 🚀 Setup and Helper Functions

import json
from datetime import datetime, timedelta
import glob
import os

# --- Helper function to round and ensure unique timestamps ---

def update_timestamps(data):
    """
    Rounds timestamps in 'gameMoves' and 'doSomethingTimestamps' to the
    nearest 500 milliseconds (0, 500) and ensures all timestamps in the
    entire file are strictly increasing.

    Args:
        data (dict): The loaded JSON data from a game file.

    Returns:
        dict: The data dictionary with updated timestamps.
    """
    all_timestamps = {}

    # Collect all timestamps from both sections into a single sorted list
    for k, v in data['gameMoves'].items():
        all_timestamps[k] = ('gameMoves', v)
    for k, v in data['doSomethingTimestamps'].items():
        all_timestamps[k] = ('doSomethingTimestamps', v)

    # Sort the items by their original timestamp string
    sorted_timestamps = sorted(all_timestamps.items())

    updated_timestamps = {}
    previous_timestamp = None

    # Iterate through the sorted list, process, and ensure uniqueness
    for original_ts_str, (key_type, value) in sorted_timestamps:
        # 1. Parse the timestamp
        # Replace 'Z' with '+00:00' for proper ISO 8601 parsing by fromisoformat
        timestamp = datetime.fromisoformat(original_ts_str.replace('Z', '+00:00'))

        # 2. Round to nearest 500 milliseconds (0 or 500)
        milliseconds = timestamp.microsecond // 1000
        remainder = milliseconds % 500

        if remainder < 250:
            milliseconds -= remainder
        else:
            milliseconds += (500 - remainder)

        # Handle overflow from 999ms to 1000ms (which is 1s)
        if milliseconds == 1000:
            timestamp += timedelta(seconds=1)
            milliseconds = 0

        rounded_timestamp = timestamp.replace(microsecond=milliseconds * 1000)

        # 3. Ensure strictly increasing uniqueness
        # If the current rounded timestamp is less than or equal to the previous,
        # increment it by 500ms.
        if previous_timestamp and rounded_timestamp <= previous_timestamp:
            rounded_timestamp = previous_timestamp + timedelta(milliseconds=500)

        previous_timestamp = rounded_timestamp

        # 4. Format the new timestamp string and store
        # Convert back to ISO format and replace '+00:00' with 'Z'
        new_ts_str = rounded_timestamp.isoformat().replace('+00:00', 'Z')

        updated_timestamps[new_ts_str] = (key_type, value)

    # 5. Rebuild the 'gameMoves' and 'doSomethingTimestamps' dictionaries
    new_game_moves = {}
    new_do_something_timestamps = {}

    for new_ts_str, (key_type, value) in updated_timestamps.items():
        if key_type == 'gameMoves':
            new_game_moves[new_ts_str] = value
        elif key_type == 'doSomethingTimestamps':
            new_do_something_timestamps[new_ts_str] = value

    data['gameMoves'] = new_game_moves
    data['doSomethingTimestamps'] = new_do_something_timestamps

    return data

# --- Main processing function ---

def process_files(file_list, output_prefix='_updated'):
    """
    Processes a list of JSON files to update timestamps and saves the
    result to a new file with the specified prefix.
    """
    print(f"--- Starting file processing for {len(file_list)} files ---")

    for input_filepath in file_list:
        try:
            # 1. Determine the output filename
            directory = os.path.dirname(input_filepath)
            filename_with_ext = os.path.basename(input_filepath)
            name, ext = os.path.splitext(filename_with_ext)

            # Insert the '_updated' prefix before the final numeric part if present
            # e.g., 'File_1.json' -> 'File_updated_1.json'
            # Find the last underscore and everything after it (e.g., '_1')
            last_underscore_index = name.rfind('_')
            if last_underscore_index != -1 and name[last_underscore_index+1:].isdigit():
                base_name = name[:last_underscore_index]
                suffix = name[last_underscore_index:]
                new_name = f"{base_name}{output_prefix}{suffix}"
            else:
                new_name = f"{name}{output_prefix}"

            output_filepath = os.path.join(directory, f"{new_name}{ext}")

            print(f"\nProcessing: {input_filepath}")
            print(f"Output to: {output_filepath}")

            # 2. Load the data
            with open(input_filepath, 'r') as f:
                data = json.load(f)

            # 3. Update the timestamps
            updated_data = update_timestamps(data)

            # 4. Save the new data
            with open(output_filepath, 'w') as f:
                json.dump(updated_data, f, indent=4)

            print("Successfully updated and saved.")

        except Exception as e:
            print(f"An error occurred while processing {input_filepath}: {e}")

# --- Execution Cell ---

# Define the pattern to find all relevant JSON files.
# Assuming the structure is similar to the file you provided:
# e.g., '../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_1.json'
# You may need to adjust the glob pattern based on your directory structure.
file_pattern = '../output/all/*/*.json'

# Get the list of all files to process
all_files_to_process = glob.glob(file_pattern)

# Execute the main function
process_files(all_files_to_process)

--- Starting file processing for 165 files ---

Processing: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_1.json
Output to: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_updated_1.json
Successfully updated and saved.

Processing: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_10.json
Output to: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_updated_10.json
Successfully updated and saved.

Processing: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_2.json
Output to: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_updated_2.json
Successfully updated and saved.

Processing: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_3.json
Output to: ../output/all\Four of the Same Type (BASIC)\Four of the Same Type (BASIC)_updated_3.json
Successfully updated and saved.

Processing: ../output/all\Four of the Same Typ